## Ejercicio 01

1. Obtener todas las cartas de la API.
1. Guardar la respuesta en un archivo `.json()`
1. La key `colors` contiene un value que es una lista. Filtrar las cartas que contengan `"Green"` en esa lista.
1. Generar una base de datos con [sqlite3](https://docs.python.org/3/library/sqlite3.html) para almacenar estas cartas.
1. Las columnas que debemos crear son (entre paréntesis está el nombre de la key que tienen en el diccionario):
    * nombre (`name`)
    * ** multiverse_id (`multiverseid`)  <== queda eliminado a menos que queraís hacer la parte extra
    * tipo (`type`)
    * **  url_imagen (`imageUrl`) <== queda eliminado a menos que queraís hacer la parte extra
    * rareza (`rarity`)
    * **Extra**: en las cartas verdes que NO tienen `multiverseid`, crearlo y darle el valor `0`
    
1. De la lista de cartas filtradas que hemos obtenido en el punto $2$, guardarlas todas en la base de datos.

**Extra 🔥**

* Crear una función que obtenga la información de la carta de la base de datos en base a un `id` o como queráis.
* En las cartas que no tengan la variable `"imageUrl"`, crearla y poner: https://picsum.photos/200/200
* Descargar la imagen de esta carta y guardarla en un archivo en el disco.
* Crear un archivo JSON que sea una lista de diccionarios con el siguiente formato:

```python
[
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    .
    .
    .
]
```

Por ejemplo:

```python
req_img = requests.get(v[0]["imageUrl"])

with open("imagen.jpg", "wb") as f:
    f.write(req_img.content)
```

In [ ]:
import requests
import json

import sqlite3

In [ ]:
respuesta = requests.get(URL)

cartas = respuesta.json()

with open("cartas.json", "w") as f:
    f.write(json.dumps(cartas))

In [ ]:
cartas_filtradas = []

for carta in cartas:
    if "Green" in carta["colors"]:
        cartas_filtradas.append(carta)

In [ ]:
cartas_filtradas = [carta for carta in cartas if "Green" in carta["colors"]]

In [ ]:
con = sqlite3.connect("magic.db")

con.execute("CREATE TABLE cartas (nombre TEXT, tipo TEXT, rareza TEXT)")

In [ ]:
QUERY = "INSERT INTO cartas VALUES (:nombre, :tipo, :rareza)"

for carta in cartas_filtradas:
    nombre = carta["name"]
    tipo = carta["type"]
    rareza = carta["rarity"]
    parametros_query = {"nombre": nombre, "tipo": tipo, "rareza": rareza}
    con.execute(QUERY, parametros_query)
    
con.commit()

In [ ]:
con.execute("SELECT * FROM cartas").fetchall()

In [ ]:
con.execute(
    "CREATE TABLE cartas2 (nombre TEXT, tipo TEXT, rareza TEXT, multi INTEGER, url TEXT)"
)

QUERY = "INSERT INTO cartas2 VALUES (:nombre, :tipo, :rareza, :multi, :url)"

lista_parametros_query = []

for carta in cartas_filtradas:
    nombre = carta["name"]
    tipo = carta["type"]
    rareza = carta["rarity"]

    try:
        multiverse = carta["multiverseid"]
    except KeyError:
        multiverse = 0

    url = carta.get("url", "https://picsum.photos/200/200")
    parametros_query = {
        "nombre": nombre,
        "tipo": tipo,
        "rareza": rareza,
        "multi": multiverse,
        "url": url,
    }

    lista_parametros_query.append(parametros_query)

con.executemany(QUERY, lista_parametros_query)


con.commit()

In [ ]:
con.row_factory = sqlite3.Row

res = con.execute("SELECT * FROM cartas2").fetchall()

[dict(x) for x in res]

Funcion para buscar una carta por nombre

In [ ]:
def buscar_carta_nombre(nombre):

    cursor = con.execute("SELECT * FROM cartas2 WHERE nombre = ?", (nombre,))

    return cursor.fetchone()

In [ ]:
dict(buscar_carta_nombre("Commune with Nature"))

Fucion para descargar una imagen

In [ ]:
def descargar_url_imagen(url, archivo):
    """
    Descargar bytes de una URL

    Esta funcion descarga los contenidos de una URL y los guarda
    en el archivo especificado.
    Si el archivo no tiene extension, por defecto le
    añade '.jpg' al nombre.

    El archivo se guarda por defecto en la carpeta actual de ejecucion.
    """

    res = requests.get(url)

    res.raise_for_status()

    if not archivo.endswith(".jpg"):
        archivo = f"{archivo}.jpg"

    with open(archivo, "wb") as f:
        f.write(res.content)

In [ ]:
descargar_url_imagen(
    "http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=136279&type=card",
    "test.jpg",
)

In [ ]:
cartas2 = con.execute("SELECT * FROM cartas2").fetchall()

for carta in cartas2:
    carta = dict(carta)
    print(carta)

    descargar_url_imagen(carta["url"], carta["nombre"])

Crear un archivo JSON que sea una lista de diccionarios con el siguiente formato:

```python
[
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    {"nombre": nombre_carta, "url_imagen": url_imagen},
    .
    .
    .
]
```

In [ ]:
def generar_dict_carta(carta):

    name = carta["name"]
    url = carta.get("imageUrl", "https://picsum.photos/200/200")

    return {"nombre": name, "url": url}


datos = [generar_dict_carta(carta) for carta in cartas]

with open("cartas2.json", "w") as f:
    f.write(json.dumps(data))